<a href="https://colab.research.google.com/github/BernieT7/ML/blob/main/CNN_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader
from torch import nn
import torch
import torchvision

In [ ]:
train_data = datasets.FashionMNIST(                # pytorch官網 > Docs > torchvision > Datasets > image classification
  root="image",     # 儲存檔名
  train=True,      # 取得訓練集
  download=True,
  transform=ToTensor()  # 轉換成tensor
)

test_data = datasets.FashionMNIST(
  root="image",     # 儲存檔名
  train=False,      # 取得測試集
  download=True,
  transform=ToTensor()  # 轉換成tensor
)

100%|██████████| 26421880/26421880 [00:00<00:00, 114795978.12it/s]


Extracting image/FashionMNIST/raw/train-images-idx3-ubyte.gz to image/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 5700367.57it/s]

Extracting image/FashionMNIST/raw/train-labels-idx1-ubyte.gz to image/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 62447653.81it/s]


Extracting image/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to image/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 23019485.07it/s]

Extracting image/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to image/FashionMNIST/raw



In [ ]:
# 過去gradient decent時，要看過所有的資料才能進行，但如果今天的資料非常大量將會遇到執行時間過長、記憶體暫存
# 空間不足等等問題，因此處理巨量資料時，我們需要做batch分堆的動作，這樣讀資料就不用讀取所有資料，讀取該堆就好
BATCH_SIZE = 32
train_dataloader = DataLoader(
  train_data,                  # 要分堆的資料(有x(圖形)跟y(類別))
  batch_size=BATCH_SIZE,            # 分堆總數==>資料總數/分堆大小
  shuffle=True                 # 是否打亂後再分堆
)

test_dataloader = DataLoader(
  test_data,
  batch_size=BATCH_SIZE,
  shuffle=False
)

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7fadcb6d9b70>,
 <torch.utils.data.dataloader.DataLoader at 0x7fadcb6da200>)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
x_first_batch, y_first_batch = next(iter(train_dataloader))
x_first_batch.shape, y_first_batch.shape

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

In [ ]:
# 卷積層: pytorch官網 > Docs > torch.nn > Conv2d
conv_layer = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=(3, 3), stride=1, padding=1)    # out_channel數目取決於filter數目
print(conv_layer(x_first_batch[0]).shape)
# max pooling: pytorch官網 > Docs > torch.nn > MaxPool2d
maxpool = nn.MaxPool2d(kernel_size=(2, 2), stride=2, padding=0)
print(maxpool(x_first_batch[0]).shape)

torch.Size([5, 28, 28])
torch.Size([1, 14, 14])


In [ ]:
def accuracy_fn(y_pred, y_real):                           # 準確度函數
  correct_num = (y_pred==y_real).sum()
  acc = correct_num / len(y_real) * 100

  return acc

In [ ]:
def train_step(dataloader, model, cost_fn, optimizer, accuracy_fn, device):   # 訓練函數
  train_cost = 0                                  # 令cost初始為0
  train_acc = 0                                  # 令acc初始為0
  for batch, (x, y) in enumerate(dataloader):                  # 跑過所有的Batch
    x = x.to(device)                               # x連線到GPU
    y = y.to(device)                               # y連線到GPU

    model.train()

    y_pred = model(x)

    cost = cost_fn(y_pred, y)

    train_cost += cost                              # 累加cost
    train_acc += accuracy_fn(y_pred.argmax(dim=1), y)              # 累加acc

    optimizer.zero_grad()

    cost.backward()

    optimizer.step()

  train_cost /= len(train_dataloader)                       # 計算平均cost
  train_acc /= len(train_dataloader)                        # 計算平均acc

  print(f"\nTrain Cost: {train_cost:.4f}, Train Acc: {train_acc:.2f}")


def test_step(dataloader, model, cost_fn, accuracy_fn, device):          # 測試模型
  test_cost = 0                                  # 令cost初始為0
  test_acc = 0                                   # 令acc初始為0
  model.eval()
  with torch.inference_mode():
    for x, y in dataloader:                           # 跑過所有的Batch
      x = x.to(device)                             # x連線到GPU
      y = y.to(device)                             # y連線到GPU

      test_pred = model(x)

      test_cost += cost_fn(test_pred, y)                   # 累加cost
      test_acc += accuracy_fn(test_pred.argmax(dim=1), y)           # 累加acc

    test_cost /= len(test_dataloader)                      # 計算平均cost
    test_acc /= len(test_dataloader)                       # 計算平均acc

  print(f"Test Cost: {test_cost:.4f}, Test Acc: {test_acc:.2f} \n")

In [ ]:
# CNN-VGG model
class ImageClassificationModel3(nn.Module):                    # 創建模型類別
  def __init__(self, input_shape, output_shape):                #初始化
    super().__init__()
    self.block_1 = nn.Sequential(
      nn.Conv2d(in_channels=input_shape,                   # 卷積層1
          out_channels=8,
          kernel_size=(3, 3),
          stride=1,
          padding=1
      ),
      nn.ReLU(),
      nn.Conv2d(in_channels=8,                        # 卷積層2
          out_channels=8,
          kernel_size=(3, 3),
          stride=1,
          padding=1
      ),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=(2, 2),                    # 池化層
            stride=2,
            padding=0
      )
    )
    self.block_2 = nn.Sequential(
      nn.Conv2d(in_channels=8,                        # 卷積層3
          out_channels=16,
          kernel_size=(3, 3),
          stride=1,
          padding=1
      ),
      nn.ReLU(),
      nn.Conv2d(in_channels=16,                        # 卷積層4
          out_channels=16,
          kernel_size=(3, 3),
          stride=1,
          padding=1
      ),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=(2, 2),                    # 池化層2
            stride=2,
            padding=0
      )
    )
    self.classifier = nn.Sequential(
      nn.Flatten(start_dim=1, end_dim=-1),                  # 拉平
      nn.Linear(in_features=16 * 7 * 7, out_features=output_shape)    # 全連接層
    )

  def forward(self, x):                             # forward函數
    return self.classifier(self.block_2(self.block_1(x)))

In [ ]:
model_3 = ImageClassificationModel3(1, 10)
model_3.to(device)
cost_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_3.parameters(), lr=0.1)

In [ ]:
from tqdm.auto import tqdm
epochs = 10

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n-------")

  train_step(train_dataloader, model_3, cost_fn, optimizer, accuracy_fn, device)

  test_step(test_dataloader, model_3, cost_fn, accuracy_fn, device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
-------

Train Cost: 0.6310, Train Acc: 77.23
Test Cost: 0.3879, Test Acc: 86.27 

Epoch: 1
-------

Train Cost: 0.3428, Train Acc: 87.79
Test Cost: 0.3414, Test Acc: 87.70 

Epoch: 2
-------

Train Cost: 0.3067, Train Acc: 88.89
Test Cost: 0.3194, Test Acc: 88.58 

Epoch: 3
-------

Train Cost: 0.2875, Train Acc: 89.61
Test Cost: 0.3143, Test Acc: 88.64 

Epoch: 4
-------

Train Cost: 0.2746, Train Acc: 90.02
Test Cost: 0.2997, Test Acc: 89.36 

Epoch: 5
-------

Train Cost: 0.2621, Train Acc: 90.48
Test Cost: 0.2916, Test Acc: 89.55 

Epoch: 6
-------

Train Cost: 0.2561, Train Acc: 90.73
Test Cost: 0.2970, Test Acc: 89.41 

Epoch: 7
-------

Train Cost: 0.2479, Train Acc: 90.89
Test Cost: 0.3030, Test Acc: 89.22 

Epoch: 8
-------

Train Cost: 0.2434, Train Acc: 91.09
Test Cost: 0.3050, Test Acc: 89.42 

Epoch: 9
-------

Train Cost: 0.2380, Train Acc: 91.37
Test Cost: 0.2806, Test Acc: 90.02 

